In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
!nvidia-smi

import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten, Conv2DTranspose, MaxPooling2D, UpSampling2D, Add
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU, Lambda, ReLU, Concatenate

from functools import partial
import re
import numpy as np
import math, os
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
import random
import cv2
from glob import glob

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
!pip3 install cmapy

# Parameters definition

In [ ]:
BASE_GCS = 'gs://pcbs_thomas/grids/'
BASE_DATA = '/content/gdrive/MyDrive/mpi_pcbs/'

## 
edge_x = [0, 24]
edge_y = [0, 14]

grid_list = [ [16,0], [24,0], [8,7], [16,7], [8,14], [24,14] ]
#grid_list = [ [16,7] ]

params = {
  'use_layer' : 12, #Layers: 5, 10, 15, 20 -> Melhor até agora: 12
  'BATCH_SIZE' : 128,
  'RECORD_SIZE' : (256+20,256+20,3),
  'IMAGE_SIZE' : (256,256,3),
  'LATENT_DIM' : 500,

  'SEED' : 100,

  'use_bn' : True,
  'use_bn_dconv' : True,
  'lrelu_slop' : 0.2,

  'use_tpu' : True,

  'USE_TFRECORD' : False
}

def get_last_model(BASE_PATH):
  all_dir = sorted(glob(BASE_PATH + "*/"))
  return all_dir[-1]

# Initialization of TPUs 

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu is not None:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

AUTO = tf.data.experimental.AUTOTUNE

# Defining the model and load the weights

In [ ]:
def conv_block(x, filters=16, kernel=5, stride=2, transpose=False, leaky=True, slope=0.2, padding='same', bn=False, bias=True, only_conv=False):
  conv = Conv2DTranspose if transpose else Conv2D
  activation = LeakyReLU(slope) if leaky else ReLU()
  
  x = conv(filters=filters, kernel_size=(kernel,kernel), strides=stride, padding=padding, use_bias=bias)(x)
  if not only_conv:
    if bn:
      x = BatchNormalization()(x)
    x = activation(x)
  
  return x

def get_cae():
  with strategy.scope():
    n_leves_enc = 7
    n_dense = int(params['IMAGE_SIZE'][0]/(2**n_leves_enc))

    inputs = Input(shape=params['IMAGE_SIZE'], name='encoder_input')
    conv1 = conv_block(inputs, 32, bn=params['use_bn'], slope=params['lrelu_slop'])
    conv2 = conv_block(conv1, 64, bn=params['use_bn'], slope=params['lrelu_slop'])  
    conv3 = conv_block(conv2, 128, bn=params['use_bn'], slope=params['lrelu_slop'])  
    conv4 = conv_block(conv3, 128, bn=params['use_bn'], slope=params['lrelu_slop'])   
    conv5 = conv_block(conv4, 256, bn=params['use_bn'], slope=params['lrelu_slop'])   
    conv6 = conv_block(conv5, 256, bn=params['use_bn'], slope=params['lrelu_slop'])  
    conv7 = conv_block(conv6, 256, bn=params['use_bn'], slope=params['lrelu_slop']) 

    conv7_flat = Flatten()(conv7)
    fc1 = Dense(units=(n_dense*n_dense*256))(conv7_flat)
    if params['use_bn']:
      fc1 = BatchNormalization()(fc1)
    fc1 = LeakyReLU(alpha=params['lrelu_slop'])(fc1)

    fc1 = Dense(units=params['LATENT_DIM'])(fc1)
    fc1 = LeakyReLU(alpha=params['lrelu_slop'])(fc1)

    fc2 = Dense(units=(n_dense*n_dense*256))(fc1)
    if params['use_bn']:
      fc2 = BatchNormalization()(fc2)
    fc2 = LeakyReLU(alpha=params['lrelu_slop'])(fc2)
    
    z_mat = Reshape((n_dense,n_dense,256))(fc2)
    dconv0 = conv_block(z_mat, 256, transpose=True, bn=params['use_bn_dconv'], slope=params['lrelu_slop'])
    dconv1 = conv_block(dconv0, 256, transpose=True, bn=params['use_bn_dconv'], slope=params['lrelu_slop'])
    dconv2 = conv_block(dconv1, 128, transpose=True, bn=params['use_bn_dconv'], slope=params['lrelu_slop'])
    dconv3 = conv_block(dconv2, 128, transpose=True, bn=params['use_bn_dconv'], slope=params['lrelu_slop'])
    dconv4 = conv_block(dconv3, 64, transpose=True, bn=params['use_bn_dconv'], slope=params['lrelu_slop'])
    dconv5 = conv_block(dconv4, 32, transpose=True, bn=params['use_bn_dconv'], slope=params['lrelu_slop'])
    

    dconv6 = Conv2DTranspose(filters=params['IMAGE_SIZE'][2], kernel_size=(5,5), strides=2, padding='same', 
        use_bias=True, activation='sigmoid')(dconv5)

    ae_model = Model(inputs, dconv6, name="ae")
    #ae_model.summary()
  return ae_model

# Creating the TF.datasets with Data augmentation

In [ ]:
def decode_image(image):
  image = tf.image.decode_jpeg( image, channels=params['RECORD_SIZE'][2] )
  image = tf.cast(image, tf.float32) / 255.0

  image = tf.reshape(image, [*params['RECORD_SIZE']])
  return image

def decode_mask(image):
  image = tf.image.decode_jpeg( image, channels=1 )
  image = tf.cast(image, tf.float32) / 255.0

  image = tf.reshape(image, [params['RECORD_SIZE'][0], params['RECORD_SIZE'][1], 1])
  return image

def read_tfrecord(example):
  feature = {'image':tf.io.FixedLenFeature([], tf.string),
              'mask1': tf.io.FixedLenFeature([], tf.string),
              'mask2': tf.io.FixedLenFeature([], tf.string),
              'image_name': tf.io.FixedLenFeature([], tf.string)
            }

  example = tf.io.parse_single_example(example, feature)
  image = decode_image(example['image'])
  mask1 = decode_mask(example['mask1'])
  mask2 = decode_mask(example['mask2'])
  name = example['image_name']
  name = tf.cast(name, tf.string)

  return image, mask1, mask2

def get_all_cropped(image, mask1, mask2):
  init_x = 10
  init_y = 10
  if grid_x == edge_x[0]:
    init_x = 0
  else:
    init_x = 10

  if grid_y == edge_y[0]:
    init_y = 0
  else:
    init_y = 10
  return tf.image.crop_to_bounding_box(image, init_x, init_y, 256, 256), tf.image.crop_to_bounding_box(mask1, init_x, init_y, 256, 256), tf.image.crop_to_bounding_box(mask2, init_x, init_y, 256, 256)

def read_image(img_path, channels=3):
  img = tf.io.read_file(img_path)
  img = tf.image.decode_image(img, channels=channels)
  if channels == 1:
    img = 255-img
  img = tf.cast(img, tf.float32) / 255.0

  return img

def read_image_and_mask(img_path):
  input_img = read_image(img_path)

  if not tf.strings.regex_full_match(img_path, "[\w/.-]*good{1}[\w/.-]*"):
    tmp_img_path = tf.strings.regex_replace(img_path, '/test/', '/ground_truth/')
    mask1_img_path = tf.strings.regex_replace(tmp_img_path, '.png', '_mask.png')
    mask1 = mask2 = read_image(mask1_img_path, 1)
  else:
    mask1 = mask2 = read_image('./blank_image.png', 1)
    #mask1 = mask2 = tf.cast( 255*tf.ones( tf.shape(input_img)[:-1] ), tf.float32 ) / 255.0

    #mask1 = tf.reshape(mask1, [params['IMAGE_SIZE'][0], params['IMAGE_SIZE'][1], 1])
    #mask2 = tf.reshape(mask1, [params['IMAGE_SIZE'][0], params['IMAGE_SIZE'][1], 1])
    #mask1 = mask2 = tf.zeros(tf.shape(input_img)[:-1])

  return input_img, mask1, mask2

def load_dataset(filenames, use_tfrecord=True, ordered=False):
  ignore_order = tf.data.Options()
  if not ordered:
    ignore_order.experimental_deterministic = False

  if use_tfrecord:
    print("Reading from tfrecord!")
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(partial(read_tfrecord), num_parallel_calls=AUTO)
  else:
    img_blank = np.zeros((params['IMAGE_SIZE'][1],params['IMAGE_SIZE'][0],3), np.uint8)
    cv2.imwrite("./blank_image.png", img_blank)    
    print("Reading from file list!")
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(partial(read_image_and_mask), num_parallel_calls=AUTO)

  return dataset

def get_test_dataset(filenames, use_tfrecord=True):
  dataset = load_dataset(filenames, use_tfrecord=use_tfrecord)
  #dataset = dataset.shuffle(2048, seed=params['SEED'])
  dataset = dataset.batch(params['BATCH_SIZE'])
  if use_tfrecord:
    dataset = dataset.map(lambda image, mask1, mask2: get_all_cropped(image, mask1,mask2), num_parallel_calls=AUTO )
  dataset = dataset.cache()
  dataset = dataset.prefetch(AUTO)
  return dataset


#Prediction and perceptual difference



In [ ]:
def predict_perpdiff(ds_imgs, model):
  print("Prediction...")
  perp_model = tf.keras.applications.VGG19(input_shape=(224,224,3))
  modelOutputs = [ perp_model.layers[i].output for i in [params['use_layer']] ]
  perp_model = Model(perp_model.inputs, modelOutputs)


  for ds_imgs in test_dataset.take(1):
    input_imgs = ds_imgs[0].numpy()

    ## Predict the reconstructed image
    y_pred = model.predict(input_imgs)  

    ## Resize the images to the input size of VGG19
    y_true_reshape = tf.image.resize(input_imgs, (224, 224))
    y_pred_reshape = tf.image.resize(y_pred, (224, 224))
    mask1_reshape = tf.image.resize(ds_imgs[1], (224, 224))
    mask2_reshape = tf.image.resize(ds_imgs[2], (224, 224))

    ## Pass all images in the VGG19 and get the perceptual layers outputs
    perp_input = perp_model([y_true_reshape])
    perp_pred = perp_model([y_pred_reshape])

  return y_true_reshape, y_pred_reshape, perp_input, perp_pred, mask2_reshape, y_pred

In [ ]:
def compute_min_max(perp_input, perp_pred):
  print("Computing min and max...")
  max_norm = 0
  min_norm = 1000000

  for img_idx in range(perp_input.shape[0]):
    perp_img = tf.math.abs( perp_input[img_idx,:,:,0] - perp_pred[img_idx,:,:,0] )
    
    for i in range(1, perp_input.shape[3] ):
      perp_img += tf.math.abs( perp_input[img_idx,:,:,i] - perp_pred[img_idx,:,:,i] )
      max_value = tf.math.reduce_max(perp_img)
      min_value = tf.math.reduce_min(perp_img)

      if max_value > max_norm:
        max_norm = max_value
      if min_value < min_norm:
        min_norm = min_value

  print("max_norm: {}".format(max_norm))
  print("min_norm: {}".format(min_norm))

  return min_norm, max_norm

In [ ]:
def calculate_anomaly_map(perp_input, perp_pred, min_norm, max_norm):
  y_pred_all = []
  all_perp = []

  print("Calculating anomaly map")
  for img_idx in range(perp_input.shape[0]):
      perp_img = tf.math.abs( perp_input[img_idx,:,:,0] - perp_pred[img_idx,:,:,0] )
      for i in range(1, perp_input.shape[3] ):
        perp_img += tf.math.abs( perp_input[img_idx,:,:,i] - perp_pred[img_idx,:,:,i] )

      perp_img =(perp_img - min_norm) / (max_norm - min_norm)      

      perp_img = tf.image.resize(tf.expand_dims(perp_img, axis=2), (224,224))
      perp_img = tf.squeeze(perp_img)

      all_perp.append(perp_img)
      y_pred_all.append( perp_img.numpy().ravel() )
  
  return all_perp, y_pred_all

In [ ]:
def process_mask(mask_arr, min_pixel_classification=10):
  mask_invert = 1-mask_arr
  ## Generate classification labels
  mask_classification = np.array([np.sum(n) > min_pixel_classification for n in mask_invert])
  ## Adjust segmentation labels
  mask_invert = tf.reshape(mask_invert, [-1])
  mask_invert = mask_invert.numpy().astype(bool)

  return mask_classification, mask_invert

def generate_seg_auc(y_pred, mask):
  print("Ploting ROC AUC") 
  
  fpr, tpr, thresholds = roc_curve(mask, np.array(y_pred).ravel())
  roc_auc = auc(fpr,tpr)

  gmeans = np.sqrt(tpr * (1-fpr))
  ix = np.argmax(gmeans)
  th_max_rocauc = thresholds[ix]
  print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

  fig, ax = plt.subplots(1,1)
  ax.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
  ax.plot([0, 1], [0, 1], 'k--')
  ax.set_xlim([0.0, 1.0])
  ax.set_ylim([0.0, 1.05])
  ax.set_xlabel('False Positive Rate')
  ax.set_ylabel('True Positive Rate')
  ax.set_title('Segmentation ROC-AUC')
  ax.legend(loc="lower right")

  return fpr, tpr, roc_auc, th_max_rocauc

In [ ]:
def get_other_metrics(y_pred, mask, class_labels, th_step = 0.01, min_pixel_classification=10):

  print("Calculating IoU, detection ROC-AUC, F1-Score, Recall and Precision")
  all_iou = []
  
  all_precision = []
  all_recall = []
  all_f1 = []

  d_fpr = []
  d_tpr = []

  d_precision = []
  d_recall = []
  d_f1_score = []
  
  th = 1.0
  while th >= 0.0:
    ### IoU ###
    y_preds = np.array(y_pred).ravel() > th
    y_preds = y_preds.astype(bool)
    tn, fp, fn, tp = confusion_matrix(mask, y_preds).ravel()
    
    iou = tp/(tp + fn + fp)
    all_iou.append(iou)

    ### Detection ###
    y_preds_class = np.array(y_pred) > th
    y_preds_class = np.array([np.sum(n) > min_pixel_classification for n in y_preds_class])

    tnd, fpd, fnd, tpd = confusion_matrix(class_labels, y_preds_class).ravel()
    fpr_detec = fpd / (fpd+tnd)
    tpr_detec = tpd / (tpd+fnd)
    d_fpr.append(fpr_detec)
    d_tpr.append(tpr_detec)
    th -= th_step

    ## Precision ##
    s_prec = precision_score(mask, y_preds, zero_division=0)
    d_prec = precision_score(class_labels, y_preds_class, zero_division=0)
    all_precision.append(s_prec)
    d_precision.append(d_prec)

    ## Recall ##
    s_rec = recall_score(mask, y_preds)
    d_rec = recall_score(class_labels, y_preds_class)
    all_recall.append(s_rec)
    d_recall.append(d_rec)

    ## F1-Score ##
    s_f1 = f1_score(mask, y_preds)
    d_f1 = f1_score(class_labels, y_preds_class)
    all_f1.append(s_f1)
    d_f1_score.append(d_f1)

  ##########################
  ## Showing some results ##
  ##########################
  max_iou = max(all_iou)
  th_max_iou = 1-np.argmax(all_iou)*th_step
  pos_max_iou = np.argmax(all_iou)
  print("Max IoU: {:.4f} -> {:.4f}".format(max_iou,  th_max_iou))
  print("Precision, Recall, F1-Score in max IoU TH: {:.2f}, {:.2f}, {:.2f}".format(all_precision[pos_max_iou], all_recall[pos_max_iou], all_f1[pos_max_iou]))

  max_precision = np.max(all_precision)
  max_recall = np.max(all_recall)
  max_f1 = np.max(all_f1)

  print("Max Precision, Recall, F1: {:.2f}, {:.2f}, {:.2f}".format(max_precision, max_recall, max_f1))

  ##########################
  ## Generating the curve ##
  ##########################
  d_roc_auc = auc(d_fpr, d_tpr)
  fig, ax = plt.subplots(1,1)
  ax.plot(d_fpr, d_tpr, label='ROC curve (area = %0.4f)' % d_roc_auc)
  ax.plot([0, 1], [0, 1], 'k--')
  ax.set_xlim([0.0, 1.0])
  ax.set_ylim([0.0, 1.05])
  ax.set_xlabel('False Positive Rate')
  ax.set_ylabel('True Positive Rate')
  ax.set_title('Detection ROC-AUC')
  ax.legend(loc="lower right")

  return d_fpr, d_tpr, d_roc_auc, all_precision, d_precision, all_recall, d_recall, all_f1, d_f1_score, all_iou, max_iou, th_max_iou



In [ ]:
for grid_x, grid_y in grid_list:
  ## Get model weights
  MODEL_PATH_BASE = '/content/gdrive/MyDrive/pcbs_cae/training_{}-{}_s1024/'.format(grid_x, grid_y)
  
  MODEL_PATH = get_last_model(MODEL_PATH_BASE)
  model = get_cae()
  model.load_weights(MODEL_PATH + 'best_model_val_loss.h5')

  # Loading data...
  if params['USE_TFRECORD']:
    GCS_PATH = '{}grid{}-{}_s1024'.format(BASE_GCS, grid_x, grid_y)
    TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + "/test*.tfrecord")
    test_dataset = get_test_dataset(TEST_FILENAMES)
  else:
    TEST_FILENAMES = glob(BASE_DATA + "extracted/grid{}-{}_s1024/test/*/*.png".format(grid_x, grid_y))
    test_dataset = get_test_dataset(TEST_FILENAMES, False)

  # Predict all images
  y_true_reshape, y_pred_reshape, perp_input, perp_pred, mask2_reshape, y_pred = predict_perpdiff(test_dataset, model)
  
  # Get min max for normalization
  min_norm, max_norm = compute_min_max(perp_input, perp_pred)

  # Calculate perceptual anomaly map
  all_perp, y_pred_all = calculate_anomaly_map(perp_input, perp_pred, min_norm, max_norm)

  # Generate and show metrics
  mask2_classification, mask2_invert = process_mask(mask2_reshape)
  fpr, tpr, roc_auc, th_max_rocauc = generate_seg_auc(y_pred_all, mask2_invert)
  d_fpr, d_tpr, d_roc_auc, all_precision, d_precision, all_recall, d_recall, all_f1, d_f1_score, all_iou, max_iou, th_max_iou = get_other_metrics(y_pred_all, mask2_invert, mask2_classification)

  # Save the metrics results
  with open(MODEL_PATH_BASE + 'mask2_data.npy', 'wb') as f:
    ## Saving roc_curve data for future plot
    np.save(f, fpr)
    np.save(f, tpr)
    np.save(f, np.array([roc_auc]))
    ## Saving IoU
    np.save(f, np.array(all_iou))
    np.save(f, np.array([max_iou, th_max_iou]))
    ## Saving detection ROC AUC
    np.save(f, d_fpr)
    np.save(f, d_tpr)
    np.save(f, np.array([d_roc_auc]))